In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
import os
import timeit


In [2]:
from queries import Queries
from queries import concatenate
from queries import find_constant_features
from click_simulator import CascadeModel

In [9]:
# Load Queries
queries_list = []
for name in ['train.txt', 'test.txt', 'vali.txt']:
    data_dir = '../../datasets/MSLR10K/Fold1/' + name
    queries = Queries.load_from_text(data_dir)
    cls = find_constant_features(queries)
    queries.adjust(remove_features=cls, purge=True, scale=True)
    queries.save(data_dir[:-3]+'dat')
    queries_list.append(queries)
query_all = concatenate(queries_list)
query_all.save('../../datasets/MSLR10K.dat')

In [10]:
data_dir = '../../datasets/MSLR10K.dat'
queries = Queries.load(data_dir)

In [11]:
simulator = CascadeModel('perfect')

In [12]:
start = timeit.default_timer()
n_clicks = 0
n_docs = 0
iteration = int(1e5)
for i in range(iteration):
    query = queries[np.random.choice(len(queries))]
    while query.document_count() < 10 or query.relevance_scores.max() == 0:
        query = queries[np.random.choice(len(queries))]
    y =  query.relevance_scores[np.random.permutation(query.document_count())]
    click = simulator.get_click(y)
    n_clicks += sum(click)
    n_docs += len(click)

print("running time of %d sessions: %f" %(iteration, timeit.default_timer()-start))
print("number of clicks: %d" % n_clicks)
print("number of docs: %d" % n_docs)

running time of 100000 sessions: 68.644067
number of clicks: 96598
number of docs: 12403485
